In [65]:
import os
import requests
from bs4 import BeautifulSoup
import re
from unidecode import unidecode
from nltk.tokenize import word_tokenize
import nltk
from spellchecker import SpellChecker
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
from pyvi import ViTokenizer
from pyvi import ViUtils
from textblob import TextBlob  # Đảm bảo bạn đã cài đặt thư viện TextBlob và các phiên bản liên quan
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')
f = open("./Downloads/vietnamese-stopwords.txt", "r", encoding="utf-8")
#Get Stop words Dictionaries
List_StopWords=f.read().split("\n")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HELLO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HELLO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
lookup_dict = {'j':'gì','kg':'không','dt':'điện thoại','omgruồi':' ra con ruồi','vs':'với','kbh':'không bao giờ',
               'nvien':'nhân viên','nc':'nước','fvu':'phục vụ','r':'rồi','ctr':'chương trình','xongg':'xong','nv':'nhân viên','ntn':'như thế nào','đc':'được','ko':'không','k':'không','vs':'vệ sinh','bt':'bình thường','pai':'phải','p':'phải','ng':'người','ms':'mới','hq':'hàn quốc','r':'rồi','tok':'tokboki','v':'vậy','zé':'thấy','h':'giờ','hnay':'hôm nay','ko':'không','cóa':'có','mib':'mình','mik':'mình','hbua':'hôm bữa','vk':'vợ','ck':'chồng','sd':'sử dụng','wan jong':'quan trọng','wan':'quán','g':'giờ','n':'nguyễn','bthg':'bình thường','qa':'qua','ngta':'người ta'}

In [3]:

directory_path = r"./Downloads/Data3/Data3/"

# Nhập các từng dữ liệu text vào data
def xuly_data():
    data = []
    
    # List all files in the directory
    file_list = os.listdir(directory_path)
    
    for file_name in file_list:
        # Check if the file is a text file (optional)
        if file_name.endswith(".txt"):
            # Create the full file path
            file_path = os.path.join(directory_path, file_name)

            # Read the content of the file and append it to the data list
            with open(file_path, "r", encoding="utf-8") as f:
                file_data = f.read()
                data.append(file_data)
    
    return data


In [4]:
print(xuly_data())

['Ốc làm vừa ăn . Món nào cũng ngon hết . 1 tuần ăn mấy lần mà ko ngán tí nào hixhix\n', 'Ăn_ở đây được 1 lần , hôm đấy tình_cờ đi qua nên vào ăn thử . Quán bé , bún thì chán , nước_dùng nhạt_nhẽo , thái_độ của chủ quán ( mà chả hiểu có phải chủ hay ko , mình thấy ngồi múc bún ở đấy ) không_thể mê nổi , chả bao_giờ đến đây lần thứ 2 😑\n', 'Đo ̣ c review cu ̉ a foody , hi ́ hư ̉ ng tơ ́ i ăn rô ̀ i ôm 1 bu ̣ ng thâ ́ t vo ̣ ng đi vê ̀\nCơm cuô ̣ n ko chă ̣ t tay va ̀ cơm thi ̀ nha ̃ o , đô ̀ ăn chi ̉ ơ ̉ mư ́ c trung bi ̀ nh\nHơm biê ́ t sao nhưng ma ̀ ca ́ râ ́ t ko tươi va ̀ tanh , go ̣ i ra 2 mo ́ n xong hê ́ t muô ́ n ăn tiê ́ p .\nNgô ̀ i lê ̀ đươ ̀ ng kho ́ i bu ̣ i , la ̣ i đu ̣ ng đô ̀ ăn ko ngon , phu ̣ c vu ̣ châ ̣ m go ̣ i ti ́ nh tiê ̀ n ma ̀ chơ ̀ mo ̣ c rê ̃ luôn\n1 lâ ̀ n cho vi ̃ nh viê ̃ n hok bao giơ ̀ đi ăn ơ ̉ đây nư ̃ a\nDu ̀ re ̉ hơn Tokio deli thiê ̣ t , nhưng ma ̀ gia ́ tiê ̀ n na ̀ y so vơ ́ i đô ̀ ăn mă ́ c qua ́\n', 'Hôm_qua không biết quán có dán thông_báo tạ

In [5]:
data = xuly_data()

In [6]:
def lowerdata(data):
    # Chuyển văn bản thành chữ thường
    data_lower = [d.lower() for d in data]
    return data_lower

In [7]:
data_lower = lowerdata(data)

In [8]:
data_lower

['ốc làm vừa ăn . món nào cũng ngon hết . 1 tuần ăn mấy lần mà ko ngán tí nào hixhix\n',
 'ăn_ở đây được 1 lần , hôm đấy tình_cờ đi qua nên vào ăn thử . quán bé , bún thì chán , nước_dùng nhạt_nhẽo , thái_độ của chủ quán ( mà chả hiểu có phải chủ hay ko , mình thấy ngồi múc bún ở đấy ) không_thể mê nổi , chả bao_giờ đến đây lần thứ 2 😑\n',
 'đo ̣ c review cu ̉ a foody , hi ́ hư ̉ ng tơ ́ i ăn rô ̀ i ôm 1 bu ̣ ng thâ ́ t vo ̣ ng đi vê ̀\ncơm cuô ̣ n ko chă ̣ t tay va ̀ cơm thi ̀ nha ̃ o , đô ̀ ăn chi ̉ ơ ̉ mư ́ c trung bi ̀ nh\nhơm biê ́ t sao nhưng ma ̀ ca ́ râ ́ t ko tươi va ̀ tanh , go ̣ i ra 2 mo ́ n xong hê ́ t muô ́ n ăn tiê ́ p .\nngô ̀ i lê ̀ đươ ̀ ng kho ́ i bu ̣ i , la ̣ i đu ̣ ng đô ̀ ăn ko ngon , phu ̣ c vu ̣ châ ̣ m go ̣ i ti ́ nh tiê ̀ n ma ̀ chơ ̀ mo ̣ c rê ̃ luôn\n1 lâ ̀ n cho vi ̃ nh viê ̃ n hok bao giơ ̀ đi ăn ơ ̉ đây nư ̃ a\ndu ̀ re ̉ hơn tokio deli thiê ̣ t , nhưng ma ̀ gia ́ tiê ̀ n na ̀ y so vơ ́ i đô ̀ ăn mă ́ c qua ́\n',
 'hôm_qua không biết quán có dán thông_báo

In [90]:
def deldaucau(data_lower):
    # Xóa tất cả các dấu câu trừ dấu chấm "."
    data_cleaned = []
    for d in data_lower:
        # Xóa tất cả các dấu câu trừ dấu chấm "."
        cleaned_text = re.sub(r'[.,\/#!$%\^&\*;:{}=\-`~()<>""?@+]', '', d)
        data_cleaned.append(cleaned_text)
    return data_cleaned



In [91]:
data_no_punctuation = deldaucau(data_lower)

In [93]:
data_no_punctuation

['ốc làm vừa ăn  món nào cũng ngon hết  1 tuần ăn mấy lần mà ko ngán tí nào hixhix\n',
 'ăn_ở đây được 1 lần  hôm đấy tình_cờ đi qua nên vào ăn thử  quán bé  bún thì chán  nước_dùng nhạt_nhẽo  thái_độ của chủ quán  mà chả hiểu có phải chủ hay ko  mình thấy ngồi múc bún ở đấy  không_thể mê nổi  chả bao_giờ đến đây lần thứ 2 😑\n',
 'đo ̣ c review cu ̉ a foody  hi ́ hư ̉ ng tơ ́ i ăn rô ̀ i ôm 1 bu ̣ ng thâ ́ t vo ̣ ng đi vê ̀\ncơm cuô ̣ n ko chă ̣ t tay va ̀ cơm thi ̀ nha ̃ o  đô ̀ ăn chi ̉ ơ ̉ mư ́ c trung bi ̀ nh\nhơm biê ́ t sao nhưng ma ̀ ca ́ râ ́ t ko tươi va ̀ tanh  go ̣ i ra 2 mo ́ n xong hê ́ t muô ́ n ăn tiê ́ p \nngô ̀ i lê ̀ đươ ̀ ng kho ́ i bu ̣ i  la ̣ i đu ̣ ng đô ̀ ăn ko ngon  phu ̣ c vu ̣ châ ̣ m go ̣ i ti ́ nh tiê ̀ n ma ̀ chơ ̀ mo ̣ c rê ̃ luôn\n1 lâ ̀ n cho vi ̃ nh viê ̃ n hok bao giơ ̀ đi ăn ơ ̉ đây nư ̃ a\ndu ̀ re ̉ hơn tokio deli thiê ̣ t  nhưng ma ̀ gia ́ tiê ̀ n na ̀ y so vơ ́ i đô ̀ ăn mă ́ c qua ́\n',
 'hôm_qua không biết quán có dán thông_báo tạm nghỉ không  m

In [94]:
def delnumber(data_no_punctuation):
    # Xóa chữ số
    data_cleaned = []
    for d in data_no_punctuation:
        cleaned_text = re.sub(r'\b\w*\d\w*\b', ' ', d)
        data_cleaned.append(cleaned_text)
    return data_cleaned

In [95]:
data_no_numbers = delnumber(data_no_punctuation)

In [96]:
data_no_numbers

['ốc làm vừa ăn  món nào cũng ngon hết    tuần ăn mấy lần mà ko ngán tí nào hixhix\n',
 'ăn_ở đây được   lần  hôm đấy tình_cờ đi qua nên vào ăn thử  quán bé  bún thì chán  nước_dùng nhạt_nhẽo  thái_độ của chủ quán  mà chả hiểu có phải chủ hay ko  mình thấy ngồi múc bún ở đấy  không_thể mê nổi  chả bao_giờ đến đây lần thứ   😑\n',
 'đo ̣ c review cu ̉ a foody  hi ́ hư ̉ ng tơ ́ i ăn rô ̀ i ôm   bu ̣ ng thâ ́ t vo ̣ ng đi vê ̀\ncơm cuô ̣ n ko chă ̣ t tay va ̀ cơm thi ̀ nha ̃ o  đô ̀ ăn chi ̉ ơ ̉ mư ́ c trung bi ̀ nh\nhơm biê ́ t sao nhưng ma ̀ ca ́ râ ́ t ko tươi va ̀ tanh  go ̣ i ra   mo ́ n xong hê ́ t muô ́ n ăn tiê ́ p \nngô ̀ i lê ̀ đươ ̀ ng kho ́ i bu ̣ i  la ̣ i đu ̣ ng đô ̀ ăn ko ngon  phu ̣ c vu ̣ châ ̣ m go ̣ i ti ́ nh tiê ̀ n ma ̀ chơ ̀ mo ̣ c rê ̃ luôn\n  lâ ̀ n cho vi ̃ nh viê ̃ n hok bao giơ ̀ đi ăn ơ ̉ đây nư ̃ a\ndu ̀ re ̉ hơn tokio deli thiê ̣ t  nhưng ma ̀ gia ́ tiê ̀ n na ̀ y so vơ ́ i đô ̀ ăn mă ́ c qua ́\n',
 'hôm_qua không biết quán có dán thông_báo tạm nghỉ không  m

In [97]:
def tachcau(data_no_numbers):
    # Tách câu
    data_sentences = [nltk.sent_tokenize(d) for d in data_no_numbers]
    return data_sentences

In [98]:
data_sentences = tachcau(data_no_numbers)
# Gộp các câu thành một văn bản text
combined_text = " ".join([" ".join(sentences) for sentences in data_sentences])


In [99]:
combined_text

"ốc làm vừa ăn  món nào cũng ngon hết    tuần ăn mấy lần mà ko ngán tí nào hixhix ăn_ở đây được   lần  hôm đấy tình_cờ đi qua nên vào ăn thử  quán bé  bún thì chán  nước_dùng nhạt_nhẽo  thái_độ của chủ quán  mà chả hiểu có phải chủ hay ko  mình thấy ngồi múc bún ở đấy  không_thể mê nổi  chả bao_giờ đến đây lần thứ   😑 đo ̣ c review cu ̉ a foody  hi ́ hư ̉ ng tơ ́ i ăn rô ̀ i ôm   bu ̣ ng thâ ́ t vo ̣ ng đi vê ̀\ncơm cuô ̣ n ko chă ̣ t tay va ̀ cơm thi ̀ nha ̃ o  đô ̀ ăn chi ̉ ơ ̉ mư ́ c trung bi ̀ nh\nhơm biê ́ t sao nhưng ma ̀ ca ́ râ ́ t ko tươi va ̀ tanh  go ̣ i ra   mo ́ n xong hê ́ t muô ́ n ăn tiê ́ p \nngô ̀ i lê ̀ đươ ̀ ng kho ́ i bu ̣ i  la ̣ i đu ̣ ng đô ̀ ăn ko ngon  phu ̣ c vu ̣ châ ̣ m go ̣ i ti ́ nh tiê ̀ n ma ̀ chơ ̀ mo ̣ c rê ̃ luôn\n  lâ ̀ n cho vi ̃ nh viê ̃ n hok bao giơ ̀ đi ăn ơ ̉ đây nư ̃ a\ndu ̀ re ̉ hơn tokio deli thiê ̣ t  nhưng ma ̀ gia ́ tiê ̀ n na ̀ y so vơ ́ i đô ̀ ăn mă ́ c qua ́ hôm_qua không biết quán có dán thông_báo tạm nghỉ không  mà trưa khoảng gần  

In [100]:
def converting_emojis(text):
    for x in EMOTICONS_EMO:
        text = text.replace(x, "_".join(EMOTICONS_EMO[x].replace(",", "").replace(":", "").split()))
    for x in UNICODE_EMOJI:
        text = text.replace(x, "_".join(UNICODE_EMOJI[x].replace(",", "").replace(":", "").split()))
    return text

In [101]:
data_without_emojis = converting_emojis(combined_text)

In [102]:
data_without_emojis

"ốc làm vừa ăn  món nào cũng ngon hết    tuần ăn mấy lần mà ko ngán tí nào hixhix ăn_ở đây được   lần  hôm đấy tình_cờ đi qua nên vào ăn thử  quán bé  bún thì chán  nước_dùng nhạt_nhẽo  thái_độ của chủ quán  mà chả hiểu có phải chủ hay ko  mình thấy ngồi múc bún ở đấy  không_thể mê nổi  chả bao_giờ đến đây lần thứ   expressionless_face đo ̣ c review cu ̉ a foody  hi ́ hư ̉ ng tơ ́ i ăn rô ̀ i ôm   bu ̣ ng thâ ́ t vo ̣ ng đi vê ̀\ncơm cuô ̣ n ko chă ̣ t tay va ̀ cơm thi ̀ nha ̃ o  đô ̀ ăn chi ̉ ơ ̉ mư ́ c trung bi ̀ nh\nhơm biê ́ t sao nhưng ma ̀ ca ́ râ ́ t ko tươi va ̀ tanh  go ̣ i ra   mo ́ n xong hê ́ t muô ́ n ăn tiê ́ p \nngô ̀ i lê ̀ đươ ̀ ng kho ́ i bu ̣ i  la ̣ i đu ̣ ng đô ̀ ăn ko ngon  phu ̣ c vu ̣ châ ̣ m go ̣ i ti ́ nh tiê ̀ n ma ̀ chơ ̀ mo ̣ c rê ̃ luôn\n  lâ ̀ n cho vi ̃ nh viê ̃ n hok bao giơ ̀ đi ăn ơ ̉ đây nư ̃ a\ndu ̀ re ̉ hơn tokio deli thiê ̣ t  nhưng ma ̀ gia ́ tiê ̀ n na ̀ y so vơ ́ i đô ̀ ăn mă ́ c qua ́ hôm_qua không biết quán có dán thông_báo tạm nghỉ không  mà

In [103]:
def data_stdd(text):
    with open('./Downloads/standard-vietnamese.txt', 'r', encoding='utf-8') as standard_vietnamese_file:
        pairs = [line.strip().split("=")[0] + "=" + line.strip().split("=")[1] for line in standard_vietnamese_file]
        for pair in pairs:
            key = pair.split("=")[0]
            value = pair.split("=")[1]
            text = text.replace(key, value)
        return text

In [104]:
data_stand=data_stdd(data_without_emojis)

In [105]:
data_stand

"ốc làm vừa ăn  món nào cũng ngon hết    tuần ăn mấy lần mà ko ngán tí nào hixhix ăn_ở đây được   lần  hôm đấy tình_cờ đi qua nên vào ăn thử  quán bé  bún thì chán  nước_dùng nhạt_nhẽo  thái_độ của chủ quán  mà chả hiểu có phải chủ hay ko  mình thấy ngồi múc bún ở đấy  không_thể mê nổi  chả bao_giờ đến đây lần thứ   expressionless_face đọc review của foody  híhửng tới ăn rồi ôm   bụng thất vọng đi vê ̀\ncơm cuộn ko chặt tay vàcơm thìnhão  đồăn chỉởmức trung bình\nhơm biết sao nhưng màcárất ko tươi vàtanh  gọi ra   món xong hết muốn ăn tiếp \nngồi lềđường khói bụi  lại đụng đồăn ko ngon  phục vụchậm gọi tính tiền màchờmọc rễluôn\n  lần cho vĩnh viễn hok bao giờđi ăn ởđây nữa\ndùrẻhơn tokio deli thiểt  nhưng màgiátiền này so với đồăn mắc quáhôm_qua không biết quán có dán thông_báo tạm nghỉ không  mà trưa khoảng gần   chiều mưa  mình và bạn chạy tới thì không mở_cửa       buồn    hi_vọng sẽ có_thể ghé lại để thử quán này nổi_tiếng là trà sữa  nhưng lúc mới mở thì thấy ngon  bây_giờ thì gi

In [106]:
def remove_Stop_words(text):
    #remove stop words
    text_pre=" ".join(text for text in text.split() if text not in List_StopWords)
    return text_pre


In [107]:
data_no_stopwords = remove_Stop_words(data_stand)

In [108]:
data_no_stopwords

"ốc món ngon tuần mấy ko ngán tí hixhix ăn_ở hôm đấy tình_cờ đi thử quán bé bún chán nước_dùng nhạt_nhẽo thái_độ chủ quán chả chủ ko múc bún đấy không_thể mê nổi chả bao_giờ expressionless_face đọc review foody híhửng ôm bụng thất vọng đi vê ̀ cơm cuộn ko chặt vàcơm thìnhão đồăn chỉởmức trung bình hơm màcárất ko tươi vàtanh gọi món xong tiếp lềđường khói bụi đụng đồăn ko ngon phục vụchậm gọi tiền màchờmọc rễluôn vĩnh viễn hok bao giờđi ởđây dùrẻhơn tokio deli thiểt màgiátiền đồăn mắc quáhôm_qua quán dán thông_báo tạm nghỉ trưa chiều mưa chạy mở_cửa buồn hi_vọng có_thể ghé thử quán nổi_tiếng trà sữa ngon bây_giờ giá mắc chất_lượng bình_thường thái_độ phục_vụ nhân_viên cũng_như chủ quán chảnh thực_đơn quán đơn_điệu trà sữa tạm món chả đặc_sắc đồ_ăn rẻ tầm hũ tiếu ngon thậm_chí dọn thìa mẻ khách_hàng phục_vụ uống phục_vụ thân_thiện chỗ bé bóng đầu_tư quẳng la_liệt miếng nệm sạch uống ổn hơi nhân_viên phục_vụ be_coffee ko j chê pha_chế ngon hương_vị nhân_viên nhiệt_tình nhất_là cookie pha 

In [73]:
def data_std(input_text):
    words = input_text.split()
    # Get Abbreviations Words
    text_pre=""
    for word in words:
        w=word
        w = re.sub(r'[^\w\s]','',w) #Removing Punctuation
        if w.lower() in lookup_dict:
            word=lookup_dict[w]
        text_pre=text_pre + " " + word        
    return text_pre

In [118]:
Data_Preprocessing = data_std(data_no_stopwords)

In [119]:
Data_Preprocessing

" ốc món ngon tuần mấy không ngán tí hixhix ăn_ở hôm đấy tình_cờ đi thử quán bé bún chán nước_dùng nhạt_nhẽo thái_độ chủ quán chả chủ không múc bún đấy không_thể mê nổi chả bao_giờ expressionless_face đọc review foody híhửng ôm bụng thất vọng đi vê ̀ cơm cuộn không chặt vàcơm thìnhão đồăn chỉởmức trung bình hơm màcárất không tươi vàtanh gọi món xong tiếp lềđường khói bụi đụng đồăn không ngon phục vụchậm gọi tiền màchờmọc rễluôn vĩnh viễn hok bao giờđi ởđây dùrẻhơn tokio deli thiểt màgiátiền đồăn mắc quáhôm_qua quán dán thông_báo tạm nghỉ trưa chiều mưa chạy mở_cửa buồn hi_vọng có_thể ghé thử quán nổi_tiếng trà sữa ngon bây_giờ giá mắc chất_lượng bình_thường thái_độ phục_vụ nhân_viên cũng_như chủ quán chảnh thực_đơn quán đơn_điệu trà sữa tạm món chả đặc_sắc đồ_ăn rẻ tầm hũ tiếu ngon thậm_chí dọn thìa mẻ khách_hàng phục_vụ uống phục_vụ thân_thiện chỗ bé bóng đầu_tư quẳng la_liệt miếng nệm sạch uống ổn hơi nhân_viên phục_vụ be_coffee không gì chê pha_chế ngon hương_vị nhân_viên nhiệt_tình

In [122]:
# Đường dẫn đến tệp output
output_file_path = "./Downloads/Data3/Data3/Data_Preprocessing/Data_Preprocessing.txt"

# Lưu dữ liệu đã xử lý vào tệp văn bản
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write(Data_Preprocessing)

print(f"Dữ liệu đã xử lý đã được lưu vào {output_file_path}")

Dữ liệu đã xử lý đã được lưu vào ./Downloads/Data3/Data3/Data_Preprocessing/Data_Preprocessing.txt
